# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
import keras_tuner as kt
import math
import datetime, os
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from functions.common_function import *
from functions.build_tuner_model import build_tuner_model
from functions.dataset_interpolation import dataset_interpolation_sklearn
from scipy.stats import pearsonr

'''Enabling plotting of graphs just below the plotting commands'''
%matplotlib inline
'''Enabling the disply of all rows and columns within the dataframe'''
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Constant

In [2]:
num_feature = 8
cat_col = [4, 5]
num_ori_feature = num_feature - len(cat_col)
num_target = 3
bandwidth = 100
num_epochs = 10000
num_folds = 4
directory_name = "Tuning_Counter_Choudhury_Method_SKLearn_Gaussian_Kernel"

limit = pd.DataFrame({'lower' : [303, 20, 0, 2, 6, 1.5, 122, 1236, 14], \
                     'higher' : [840, 44, 17, 5, 8, 2, 408, 3240, 101], \
                     'ref' : [530, 40, 14, 3.2, 6, 1.8, np.nan, np.nan, np.nan]})

'''Importing Dataset'''
dataset = pd.read_csv("Dataset/Choudhury_Dataset.csv")

# Implementation

In [3]:
'''Dataset splitting before interpolation to not introduce data leakage '''
x_train, x_test, y_train, y_test = tt_split(dataset, 0.2, num_feature, num_target)

'''Merging of features and targets datasets'''
train_dataset = merge_x_y(x_train, y_train)
test_dataset = merge_x_y(x_test, y_test)

'''Dataset Interpolation'''
interpolated_dataset = dataset_interpolation_sklearn(train_dataset, num_ori_feature, num_target, limit)

'''Converting Categorical Data into binary representation'''
converted_dataset = convert_cat(interpolated_dataset, cat_col, num_ori_feature, num_target, [interpolated_dataset.iloc[:, 4].unique(), interpolated_dataset.iloc[:, 5].unique()])
converted_test_dataset = convert_cat(test_dataset, cat_col, num_ori_feature, num_target, [dataset.iloc[:, 4].unique(), dataset.iloc[:, 5].unique()])

'''Normalising dataset using Min Max present in the train set'''
scaler = MinMaxScaler()
scaler.fit(converted_dataset)

'''Normalisation of training and testing set'''
normalised_train_dataset = pd.DataFrame(scaler.transform(converted_dataset), columns = get_col_names(converted_dataset))
normalised_test_dataset = pd.DataFrame(scaler.transform(converted_test_dataset), columns = get_col_names(converted_dataset))

x_train = normalised_train_dataset.iloc[:, 0: num_feature]
y_train =  normalised_train_dataset.iloc[:, num_feature: num_feature + num_target]
x_test = normalised_test_dataset.iloc[:, 0: num_feature]
y_test = normalised_test_dataset.iloc[:, num_feature: num_feature + num_target]


'''Cross Validation'''
kfold =KFold(n_splits = num_folds, shuffle = True)
fold_no = 1
for train, test in kfold.split(x_train, y_train):
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    tuner = kt.BayesianOptimization(build_tuner_model, objective = 'val_loss', max_trials = 10, directory = 'keras_tuner', project_name = directory_name, overwrite = True)
    tuner.search_space_summary()
    tuner.search(x_train.iloc[train], y_train.iloc[train], epochs = num_epochs, validation_data= (x_train.iloc[test], y_train.iloc[test]))
    tuner.results_summary()
    best_hps = tuner.get_best_models()[0]
    best_hps.save(f"Model\{directory_name}\model_{fold_no}")
    print("Saved model to disk")
    fold_no += 1

Trial 10 Complete [00h 05m 03s]
val_loss: 0.00698390556499362

Best val_loss So Far: 0.005455001723021269
Total elapsed time: 01h 13m 30s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in keras_tuner\Tuning_Counter_Choudhury_Method_SKLearn_Gaussian_Kernel
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
units1: 15
num_of_layers: 1
num_of_neurons0: 15
num_of_neurons1: 8
num_of_neurons2: 2
num_of_neurons3: 2
num_of_neurons4: 2
num_of_neurons5: 2
num_of_neurons6: 2
num_of_neurons7: 2
num_of_neurons8: 2
num_of_neurons9: 2
num_of_neurons10: 2
num_of_neurons11: 2
num_of_neurons12: 2
num_of_neurons13: 2
num_of_neurons14: 15
num_of_neurons15: 2
num_of_neurons16: 9
num_of_neurons17: 2
num_of_neurons18: 15
num_of_neurons19: 2
Score: 0.005455001723021269
Trial summary
Hyperparameters:
units1: 15
num_of_layers: 1
num_of_neurons0: 15
num_of_neurons1: 2
num_of_neurons2: 2
num_of_neurons3: 2
num_of_neurons4: 2
num_of_neurons5: 2
num_of_n

#  Loading of Models and Evaluate Them

In [4]:
results = []
for i in range(1, 5):
    '''Loads the Best Model Trained using Cross Validation'''
    loaded_model = keras.models.load_model(f"Model\{directory_name}\model_{i}")

    print("Loaded model from disk")

    '''Compilation of the model with its corresponding weights, followed by the evaluation of the model using test set'''
    loaded_model.compile(loss = 'MeanAbsoluteError',\
                        optimizer = 'SGD',\
                        metrics = [tf.keras.metrics.MeanSquaredError(),\
                        tf.keras.metrics.RootMeanSquaredError()])
    result = loaded_model.evaluate(x_test, y_test, batch_size = 128)
    print(f"Mean Absolute Error for model {i}(Loss): ", result[0])
    results.append(result[0])

'''Provide average score'''
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(results)):
    print('------------------------------------------------------------------------')
    print(f'> Iteration {i+1} - MAE: {results[i]}') 
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> MAE: {np.mean(results)} - Standard Deviation: {np.std(results)}')
print('------------------------------------------------------------------------')   

Loaded model from disk
1/1 [==============================] - 0s 2ms/step - loss: 0.6493 - mean_squared_error: 1.2692 - root_mean_squared_error: 1.1266
Mean Absolute Error for model 1(Loss):  0.6493167281150818
Loaded model from disk
1/1 [==============================] - 0s 2ms/step - loss: 0.6473 - mean_squared_error: 1.2709 - root_mean_squared_error: 1.1274
Mean Absolute Error for model 2(Loss):  0.6472587585449219
Loaded model from disk
1/1 [==============================] - 0s 2ms/step - loss: 0.6646 - mean_squared_error: 1.2721 - root_mean_squared_error: 1.1279
Mean Absolute Error for model 3(Loss):  0.6646442413330078
Loaded model from disk
1/1 [==============================] - 0s 2ms/step - loss: 0.6391 - mean_squared_error: 1.2441 - root_mean_squared_error: 1.1154
Mean Absolute Error for model 4(Loss):  0.6390843987464905
------------------------------------------------------------------------
Score per fold
--------------------------------------------------------------------

# Visualisation of Predictions using Best Model from Cross Validation

In [5]:
best_model_index = results.index(min(results)) + 1

'''Loads the Best Model Trained using Cross Validation'''
loaded_model = keras.models.load_model(f"Model\{directory_name}\model_{best_model_index}")

prediction = pd.DataFrame(loaded_model.predict(x_test), columns = get_col_names(y_test))

'''Preparation to Rescale target values'''
min_y = dataset.iloc[:, num_ori_feature: num_ori_feature + num_target].min().to_list()
max_y = dataset.iloc[:, num_ori_feature: num_ori_feature + num_target].max().to_list()

corr_list = []
'''Tabulating the differences of Expected and Predictions made by the ANN Model'''
for i in range(len(x_test)):
    '''Rescaling of normalised data'''
    expected = pd.DataFrame(inverse_transform(y_test.iloc[i].to_list(), max_y, min_y))
    predicted = pd.DataFrame(inverse_transform(prediction.iloc[i].to_list(), max_y, min_y))
    comparison_df = pd.concat([expected, predicted], axis = 1)
    comparison_df.columns = ['Expected', 'Predicted']
    comparison_df.index = get_col_names(y_test)
    display(comparison_df.style.set_caption(f"Element {i + 1}"))
    corr, _ = pearsonr(expected.iloc[:, 0].tolist(), predicted.iloc[:, 0].tolist())
    corr_list.append(corr)
    
'''Provide average score'''
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(corr_list)):
    print('------------------------------------------------------------------------')
    print(f'> Iteration {i+1} - Pearson Correlation: {corr_list[i]}') 
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Average Pearsons Correlation: {np.mean(corr_list)} - Standard Deviation: {np.std(corr_list)}')
print('------------------------------------------------------------------------')   

,Expected,Predicted
y_1,256.354529,209.554787
y_2,1953.649646,1865.561867
y_3,39.379782,35.865465


,Expected,Predicted
y_1,94.032827,230.028237
y_2,2328.182780,2025.354198
y_3,46.892537,34.977672


,Expected,Predicted
y_1,233.399945,219.774484
y_2,1993.922026,2031.319387
y_3,44.388285,42.826582


,Expected,Predicted
y_1,136.662769,217.648729
y_2,-817.090095,2008.718304
y_3,23.102147,30.000000


------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Iteration 1 - Pearson Correlation: 0.9998452742432522
------------------------------------------------------------------------
> Iteration 2 - Pearson Correlation: 0.9974859257193649
------------------------------------------------------------------------
> Iteration 3 - Pearson Correlation: 0.9999716169527251
------------------------------------------------------------------------
> Iteration 4 - Pearson Correlation: -0.9810069605290745
------------------------------------------------------------------------
Average scores for all folds:
> Average Pearsons Correlation: 0.5040739640965668 - Standard Deviation: 0.8574124431439363
------------------------------------------------------------------------
